<a href="https://colab.research.google.com/github/Niranjan0311/sa_2025/blob/main/notebookb9d13fd9b5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
ratnpandey_summer_analytics_ndvi_data_path = kagglehub.dataset_download('ratnpandey/summer-analytics-ndvi-data')

print('Data source import complete.')


In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)




import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('/kaggle/input/summer-analytics-ndvi-data/hacktrain.csv')
test = pd.read_csv('/kaggle/input/summer-analytics-ndvi-data/hacktest.csv')

print("Train shape:", train.shape)
print("Test shape:", test.shape)

ndvi_cols = [col for col in train.columns if '_N' in col]

train[ndvi_cols] = train[ndvi_cols].interpolate(axis=1, limit_direction='both')
test[ndvi_cols] = test[ndvi_cols].interpolate(axis=1, limit_direction='both')


def extract_features(df):
    f = pd.DataFrame()
    f['mean'] = df[ndvi_cols].mean(axis=1)
    f['std'] = df[ndvi_cols].std(axis=1)
    f['min'] = df[ndvi_cols].min(axis=1)
    f['max'] = df[ndvi_cols].max(axis=1)
    f['range'] = f['max'] - f['min']
    f['skew'] = df[ndvi_cols].skew(axis=1)
    f['first'] = df[ndvi_cols].iloc[:, 0]
    f['last'] = df[ndvi_cols].iloc[:, -1]
    f['trend'] = f['last'] - f['first']
    return f

X_train = extract_features(train)
X_test = extract_features(test)
y_train = train['class']
test_ids = test['ID']


le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)

model = make_pipeline(StandardScaler(), LogisticRegression(multi_class='multinomial', solver='saga', max_iter=1000))
model.fit(X_train, y_train_enc)


preds = model.predict(X_test)
pred_labels = le.inverse_transform(preds)

submission = pd.DataFrame({'ID': test_ids, 'class': pred_labels})
submission.to_csv('submission.csv', index=False)
print("✅ Saved submission.csv")


/kaggle/input/summer-analytics-ndvi-data/hacktest.csv
/kaggle/input/summer-analytics-ndvi-data/hacktrain.csv
Train shape: (8000, 30)
Test shape: (2845, 29)
✅ Saved submission.csv
